# Topic Modeling Assessment Project

Welcome to your Topic Modeling Assessment! For this project you will be working with a dataset of over 400,000 quora questions that have no labeled cateogry, and attempting to find optimal number of cateogries to assign these questions to. The .csv file of these text questions can be found in the NLP folder.


#### Task: Import pandas and read in the quora_questions.csv file.

In [49]:
import pandas as pd
import numpy as np

import random

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

/Users/JacquesPierre/anaconda3/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
/Users/JacquesPierre/anaconda3/lib/python3.7/site-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


In [65]:
df = pd.read_csv('data/quora_questions.csv')
npr = df.iloc[:10000]

In [66]:
print(npr.shape)
npr.head()

(10000, 1)


,Question
0,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,How can I increase the speed of my internet co...
3,Why am I mentally very lonely? How can I solve...
4,"Which one dissolve in water quikly sugar, salt..."


# Preprocessing

#### Task: Create a vectorized document term matrix. 

- How do you want to clean up your text with regards to stopwords, special characters, and other situations.
- Using a Countvectorizer versus a TFIDFvectorizer
- You may want to explore the max_df and min_df parameters. 


In [67]:
import re 
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

In [68]:
def spacy_tokenizer(text):
    # remove html tags from all of the text before processing
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    # Creating our token object, which is used to create documents with linguistic annotations.
    # we disabled the parser and ner parts of the pipeline in order to speed up parsing
    mytokens = nlp(cleantext, disable=['parser', 'ner'])

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [69]:
from sklearn.feature_extraction.text import CountVectorizer

In [70]:
cv = CountVectorizer(tokenizer=spacy_tokenizer, max_df=0.70, min_df=100, stop_words='english')

In [71]:
dtm = cv.fit_transform(npr['Question'])
dtm

<10000x34 sparse matrix of type '<class 'numpy.int64'>'
	with 6711 stored elements in Compressed Sparse Row format>

# LDA Modelling

#### TASK: Using Scikit-Learn create an instance of LDA. 

- You can manually run and tune your model, then evaluate the resulting clusters. 
- Or you can use gridsearch to try and identify the best number of topics to use. 


In [9]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

In [ ]:
# Define Search Param
param_grid = {
    'n_components': [15, 20, 25],
    'learning_decay': [0.1, 0.5, 0.9]
}

# Init the Model
lda = LatentDirichletAllocation(max_iter=25)

# Init Grid Search Class
lda_gs_model = GridSearchCV(lda, param_grid=param_grid, cv=3, verbose=2, n_jobs = -1)

# Do the Grid Search
lda_gs_model.fit(dtm)

print(lda_gs_model)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [ ]:
# Best Model
best_lda_model = lda_gs_model.best_estimator_

# Model Parameters
print("Best Model's Params: ", lda_gs_model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", lda_gs_model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(dtm))

#### Task: Evaluate the different models you have run and determine which model you think determines the best clusters.  


The evaluation part could invlove:
- Printing out the top 15 most common words for each of the topics and seeing if they make sense.
- Using the perplexity and log-likelihoood scores.
- Using the pyLDAvis tool to investigate the different clusters. 

In [ ]:
for index,topic in enumerate(best_lda_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

In [ ]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(dtm)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(npr))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

In [ ]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, dtm, cv)
panel

#### TASK: Add a new column to the original quora dataframe that labels each question into one of the topic categories.

In [ ]:
npr.head()

In [ ]:
topic_results = best_lda_model.transform(dtm)

In [ ]:
npr['Topic'] = topic_results.argmax(axis=1)

In [ ]:
npr.head(10)

# Great job!